In [1]:
from speechbrain.augment.time_domain import AddNoise
import os
import pandas as pd
from datasets import load_dataset, concatenate_datasets
import torchaudio
import torch
import mutagen
from mutagen.wave import WAVE
from IPython.display import Audio as AudioPlayer
import IPython.display as ipd

/workspace2/cvlachos/projects/Cross_Decoder/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Add noise


In [2]:
items1 = [os.path.join('/scratch/cvlachos/MUSAN/musan/noise/free-sound',i) for i in os.listdir('/scratch/cvlachos/MUSAN/musan/noise/free-sound') if i.endswith('.wav')]
items2 = [os.path.join('/scratch/cvlachos/MUSAN/musan/noise/sound-bible',i) for i in os.listdir('/scratch/cvlachos/MUSAN/musan/noise/sound-bible') if i.endswith('.wav')]
items = items1+items2

df = pd.DataFrame({'ID':[i for i in range(len(items))], 'duration':items, 'wav':items, 'wav_format':['wav' for i in range((len(items)))], 'wav_opts':['' for i in range((len(items)))]})

df.duration = df.duration.apply(lambda x : 5.)
df.to_csv("noise_list.csv", index=False)

In [3]:
noisifier = AddNoise(
    csv_file="noise_list.csv",
    snr_low=0,
    snr_high=15,
    pad_noise=True,
    normalize=True,
)

In [4]:
data_cache_folder = '/scratch/cvlachos/LibriSpeech/'

train_librispeech_100 = load_dataset("openslr/librispeech_asr", "clean", split='train.100', cache_dir=data_cache_folder)
clean = torch.tensor(train_librispeech_100[0]['audio']['array'])

In [5]:
noisy = noisifier(clean.unsqueeze(0), torch.ones(1))


In [12]:
noisy[0].numpy()

array([-0.00751437, -0.00023172,  0.01205858, ..., -0.00992688,
       -0.00743663, -0.00559347])

In [6]:
ipd.display(AudioPlayer(noisy,rate=22_000))

Create batches

In [3]:
data_cache_folder = '/scratch/cvlachos/LibriSpeech/'


train_librispeech_100 = load_dataset("openslr/librispeech_asr", "clean", split='train.100', cache_dir=data_cache_folder)


In [ ]:
def compute_length(example):
    audio = example["audio"]
    length_seconds = len(audio["array"]) / audio["sampling_rate"]
    example["length_seconds"] = length_seconds
    return example

# Apply transformation
ds_with_lengths = train_librispeech_100.map(compute_length)

#ds_sorted_desc = ds_with_lengths.sort("length_seconds")

Map:   0%|          | 0/28539 [00:00<?, ? examples/s]

Map:  10%|█         | 2991/28539 [07:49<01:36, 265.34 examples/s]  